Created by: Akshay Chougule

Date: 12 Aug 2024 

## 1 Story Generation

In [1]:
#!pip install -U langchain-ollama

In [2]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1")

In [3]:
response = llm.invoke("Can you write a very short story of 7 sentences for 3 year old kid?")
print(response)

Here is a very short story:

Benny the bear loved to play outside. One sunny day, Benny went on an adventure in the woods. He saw a bright yellow flower and picked it to give to his mom. When he got home, Mommy Bear said "Oh, I love this flower!" She put it in a vase and they both smiled at it together. Benny was happy that he could make his mommy smile. Then Benny went outside to play some more!


In [4]:
type(response)

str

In [6]:
temp = response.replace('Here is a very short story:\n\n','')
temp 

'Benny the bear loved to play outside. One sunny day, Benny went on an adventure in the woods. He saw a bright yellow flower and picked it to give to his mom. When he got home, Mommy Bear said "Oh, I love this flower!" She put it in a vase and they both smiled at it together. Benny was happy that he could make his mommy smile. Then Benny went outside to play some more!'

In [7]:
import re
sentences = re.split(r' *[\.\?][\'"\)\]]* *', temp)

In [8]:
sentences

['Benny the bear loved to play outside',
 'One sunny day, Benny went on an adventure in the woods',
 'He saw a bright yellow flower and picked it to give to his mom',
 'When he got home, Mommy Bear said "Oh, I love this flower!" She put it in a vase and they both smiled at it together',
 'Benny was happy that he could make his mommy smile',
 'Then Benny went outside to play some more!']

### 2 Text to Speech

In [14]:
# !pip install transformers
# !pip install gradio
# !pip install timm
# !pip install timm
# !pip install inflect
# !pip install phonemizer

In [9]:
from transformers.utils import logging

logging.set_verbosity_error()

In [10]:
from transformers import pipeline

narrator = pipeline("text-to-speech",
                    model="kakao-enterprise/vits-ljs")

/home/ubuntu/anaconda3/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
temp

'Benny the bear loved to play outside. One sunny day, Benny went on an adventure in the woods. He saw a bright yellow flower and picked it to give to his mom. When he got home, Mommy Bear said "Oh, I love this flower!" She put it in a vase and they both smiled at it together. Benny was happy that he could make his mommy smile. Then Benny went outside to play some more!'

In [12]:
text = temp

In [13]:
narrated_text = narrator(temp)

In [14]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(narrated_text["audio"][0],
             rate=narrated_text["sampling_rate"])

## 3 Text to Video

In [15]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

To avoid PEFT backend required error:

(As of 1 April 2024) Ensure your library has the following versions:

    peft >= 0.6.0
    transformers >= 4.34.0


In [16]:
import transformers
transformers.__version__

'4.38.2'

In [17]:
#!pip install peft
import peft
peft.__version__

'0.12.0'

In [18]:
adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.set_adapters(["lcm-lora"], [0.8])

pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/envs/py311/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [19]:
sentences[0]

'Benny the bear loved to play outside'

In [20]:
len(sentences)

6

In [25]:
# if cuda is out of memory then
# do nvidia-smi to check processed running
# use the process if (PID) to kill the process with
# sudo kill -5853 PID

In [21]:
def sentence_to_gif(sentences):
    for i in range(len(sentences)):
        output = pipe(
            prompt=f'{sentences[i]}, low resolution cartoon',
            negative_prompt="bad quality, worse quality, low resolution",
            num_frames=16,
            guidance_scale=2.0,
            num_inference_steps=6,
            generator=torch.Generator("cpu").manual_seed(0),
        )
        frames = output.frames[0]
        export_to_gif(frames, f'take2_scene_{i}.gif')

In [26]:
sentence_to_gif(sentences=sentences)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [23]:
import gc
torch.cuda.empty_cache()
gc.collect()

2401

In [8]:
#!pip install ipywidgets

In [27]:
import ipywidgets as widgets
gif = 'take2_scene0.gif'
display(widgets.HTML(f'<img src="{gif}" width="750" align="center">'))

HTML(value='<img src="take2_scene0.gif" width="750" align="center">')